# **Модуль А.** Анализ и предобработка данных

## Импортирование библиотек

In [ ]:
# для работы с файловой системой
import os

# для работы с раширением tar
import tarfile

# для работы с датафреймами
import pandas as pd

# для конвертации mp3 в wav
from pydub import AudioSegment

# для анализа звуковых дорожек

## Загрузка данных

In [6]:
# создаем папку для данных
os.makedirs('Data', exist_ok=True)

# получаем пути к архивам с данными
cv_corpus = '../Data/cv-corpus-21.0-delta-2025-03-14-ru.tar.gz'
ljspeech = '../Data/LJSpeech-1.1.tar.bz2'
ru_ru = '../Data/ru_RU.tgz'

Открываем первый архив

In [8]:
# открываем первый архив используя библиотеку tarfile
with tarfile.open(cv_corpus, 'r:gz') as tar:
    # распаковываем архив
    tar.extractall('Data/cv_corpus')

In [11]:
# получаем путь к папке с записями
clips_path = 'Data/cv_corpus/cv-corpus-21.0-delta-2025-03-14/ru/clips'
# получаем кол-во записей в папке
print(f'Кол-во записей в первой папке: {len(os.listdir(clips_path))}')

Кол-во записей в первой папке: 3034


Общее кол-во файлов с записями - `3034`